In [2]:
import os
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
from ultralytics import YOLO
import cv2


In [3]:

# Ver versión de PyTorch
print(f"Versión de PyTorch: {torch.__version__}")

# Ver disponibilidad de CUDA
print(f"CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Nombre de la GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA no está disponible.")


Versión de PyTorch: 2.5.1+cu124
CUDA disponible: True
Nombre de la GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [ ]:
# Configuración del dispositivo (GPU o CPU)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parámetros del modelo y entrenamiento
NUM_CLASSES = 10  # Número de equipos de F1
LEARNING_RATE = 0.0005
BATCH_SIZE = 8  # Ajusta según la memoria de la GPU
NUM_WORKERS = 8  # Aumentar si tienes más núcleos en la CPU
NUM_EPOCHS = 50 # O más, dependiendo del rendimiento
AUGMENTATION = True  # Hacer augmentación de datos




In [5]:
import cv2
import numpy as np

# Verificar si las imágenes se leen correctamente
train_image = cv2.imread("../f1-strategy/f1-dataset/train/images/1-2023-Brazilian-GP-FP1-20_jpg.rf.2f3d25867c8e1661f6070ae5a84c6dd4.jpg")
if train_image is None:
    print("Error al leer la imagen")
else:
    print("Imagen cargada correctamente: ", train_image.shape)


Imagen cargada correctamente:  (640, 640, 3)


In [6]:
import cv2
import os

# Definir la carpeta de imágenes
image_folder = "C:/Users/victo/Desktop/Documents/Tercer año/Segundo Cuatrimestre/Finales/f1-strategy/f1-dataset/train/images"

# Lista para almacenar los tamaños de las imágenes
sizes = set()

# Verificar el tamaño de cada imagen
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)
    img = cv2.imread(image_path)
    
    if img is not None:
        img_size = img.shape  # (height, width, channels)
        sizes.add(img_size)
    else:
        print(f"Error al cargar la imagen: {image_name}")

# Comprobar si todas las imágenes tienen el mismo tamaño
if len(sizes) == 1:
    print("Todas las imágenes tienen el mismo tamaño.")
else:
    print("Hay imágenes con tamaños diferentes. Los tamaños encontrados son:")
    for size in sizes:
        print(size)


Todas las imágenes tienen el mismo tamaño.


In [ ]:
# Cargar el modelo YOLOv8 Nano
from torch.optim import Adam
model = YOLO("yolov8m.pt")



model.train(
    data="../f1-strategy/f1-dataset/data.yaml",
    epochs=NUM_EPOCHS,
    batch=BATCH_SIZE,
    optimizer="SGD",         # Puedes probar también con "AdamW" para un optimizador adaptativo
    lr0=LEARNING_RATE,
    momentum=0.9,
    warmup_epochs=5,         # Menos epochs de warmup para entrar más rápido en el entrenamiento efectivo
    weight_decay=0.0001,
    label_smoothing=0.1,
    degrees=15.0,            # Rotación máxima reducida a 15° para evitar transformaciones demasiado agresivas
    mixup=0.15,              # Mezcla de imágenes de forma moderada
    copy_paste=0.05,         # Menor intensidad para no alterar en exceso la composición real de la imagen
    hsv_h=0.015,             # Cambios sutiles en el tono para mantener la naturalidad
    box=7.5,
    cls=1.5,
    patience=10,             # Más paciencia para que el scheduler pueda adaptarse y reducir la lr si es necesario
    pretrained=True,
    project='../f1-strategy/yolo-files/runs',
    name='train_v2',
)













New https://pypi.org/project/ultralytics/8.3.78 available  Update with 'pip install -U ultralytics'
WARNING  'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=../f1-strategy/f1-dataset/data.yaml, epochs=40, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=../f1-strategy/yolo-files/runs, name=train_v25, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, strea

train: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\train\labels.cache... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


Plotting labels to ..\f1-strategy\yolo-files\runs\train_v25\labels.jpg... 
optimizer: SGD(lr=0.0005, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0001), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to ..\f1-strategy\yolo-files\runs\train_v25
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         20         29      0.642      0.721      0.748      0.586



40 epochs completed in 0.594 hours.
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_v25\weights\last.pt, 52.0MB
Optimizer stripped from ..\f1-strategy\yolo-files\runs\train_v25\weights\best.pt, 52.0MB

Validating ..\f1-strategy\yolo-files\runs\train_v25\weights\best.pt...
Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  7.00it/s]


                   all         20         29      0.818      0.552      0.765       0.61
           Kick Sauber          4          6          1      0.399      0.667      0.485
          Racing Bulls          1          1      0.811          1      0.995      0.697
                Alpine          4          5          1      0.381      0.659      0.519
               Ferarri          3          4          1      0.634       0.87      0.745
                  Haas          1          1      0.881          1      0.995      0.895
               Mclaren          5          6      0.937      0.333       0.75      0.587
              Mercedes          3          3      0.917      0.667      0.806      0.675
              Williams          2          3          0          0      0.373      0.279
Speed: 2.2ms preprocess, 8.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_v25


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024C2C383D90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

### Model With this paremeters and no optimizer:
#### Configuración del dispositivo (GPU o CPU)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Parámetros del modelo y entrenamiento
NUM_CLASSES = 10  # Número de equipos de F1
BATCH_SIZE = 8
NUM_WORKERS = 4
NUM_EPOCHS = 10
LEARNING_RATE = 0.005

Mal rendimiento, pasando a nuevos valores y reentrenamiento.

### Aun mal rendimiento, Pasando a un modelo mas complejo, del nano al m.

### Nuevo modelo tras data augmentation
Después de aumentar las imágenes a 200 por clase, rendimiento del 70% obtenido en todas las clases. 
Sí, recuerdo los resultados anteriores que compartiste. Por ejemplo, tenías:

Métricas generales:

mAP50: 0.772
mAP50-95: 0.449
Precisión (P): 0.733
Recall (R): 0.615
Clases problemáticas y sus mAP50:

Williams: 0.487
Kick Sauber: 0.619 (con un recall notablemente bajo, alrededor de 0.443)
Alpine: 0.556
Clases con buenos resultados:

Racing Bulls y Haas: mAP50 > 0.99
Mclaren y Mercedes: mAP50 > 0.88
Tiempo de inferencia: 8.1 ms por imagen

Williams y Sauber con muchos falsos positivos, por lo que:

- Probamos con nuevos hiperparámetros y aumentamos las epocas.

### Modelo v3
De momento, menos gb de vram usados. 


In [8]:
model.val()

Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.11s/it]


                   all         20         29      0.819      0.552      0.765      0.611
           Kick Sauber          4          6          1        0.4      0.667      0.485
          Racing Bulls          1          1      0.811          1      0.995      0.697
                Alpine          4          5          1      0.381      0.659      0.521
               Ferarri          3          4          1      0.634       0.87      0.745
                  Haas          1          1      0.882          1      0.995      0.895
               Mclaren          5          6      0.939      0.333       0.75      0.587
              Mercedes          3          3      0.918      0.667      0.806      0.676
              Williams          2          3          0          0      0.373      0.279
Speed: 2.9ms preprocess, 35.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_v252


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024C2E1C4160>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

Comparación y Conclusión
Mejora Global:
La nueva configuración muestra una mejora en el mAP50 (0.818 vs 0.772) y en el mAP50-95 (0.552 vs 0.449), lo que indica que el modelo está detectando con mayor precisión y que sus cajas se ajustan mejor a los objetos reales.

Precisión:
La precisión global ha subido de 0.733 a 0.765, lo que es un buen indicativo de que las predicciones correctas han aumentado.

Recall:
El recall se mantiene prácticamente igual (0.615 vs 0.61), lo que sugiere que el número de falsos negativos no varió significativamente.
Sin embargo, la mejora en mAP50-95 implica que las detecciones precisas han mejorado, aunque quizá se deba trabajar estrategias adicionales para las clases problemáticas (como Kick Sauber, Alpine y Williams).

Hiperparámetros:
El uso de una tasa de aprendizaje variable (con warmup y decaimiento), junto con ajustes más moderados en la intensidad de las transformaciones (reduciendo grados de rotación, mixup y copy_paste) parecen haber contribuido a esta mejora global.

Consideraciones:
Las clases con peor desempeño (por ejemplo, Williams) siguen siendo un reto. Podría ser interesante explorar aumentos específicos o técnicas de fine-tuning para esas clases en particular.

#### Proóximos pasos.

Aumentar a 50 epochs.
Hace fine tuning de las clases problematicas.

In [10]:
model.save("../f1-strategy/weights/f1_detection_best.pt")

In [16]:
model_fine_tuned = YOLO("../f1-strategy/weights/f1_detection_best.pt")  

# model_fine_tuned.train(
#     data="../f1-strategy/f1-dataset/data.yaml",
#     epochs=NUM_EPOCHS,
#     batch=BATCH_SIZE,
#     optimizer="AdamW",
#     lr0=LEARNING_RATE / 2,  # Aprendizaje más lento para no sobreajustar
#     momentum=0.9,
#     weight_decay=0.0001,
#     label_smoothing=0.1,
#     degrees=10.0,
#     mixup=0.2,
#     patience=10,
#     pretrained=True,  # Importante para mantener el conocimiento previo
#     project='../f1-strategy/yolo-files/runs',
#     name='train_finetune'
# )


model_fine_tuned.train(
    data="../f1-strategy/f1-dataset/data.yaml",
    epochs=NUM_EPOCHS,
    batch=BATCH_SIZE,
    optimizer="AdamW",
    lr0=LEARNING_RATE / 2,  # Mantener un aprendizaje más lento para no sobreajustar
    momentum=0.9,
    weight_decay=0.0001,  # Mantener el ajuste original para evitar sobreajuste
    label_smoothing=0.1,
    degrees=10.0,
    mixup=0.2,
    patience=10,  # Paciencia ajustada a 10
    pretrained=True,  # Mantener el conocimiento previo
    project='../f1-strategy/yolo-files/runs',
    name='train_finetune',
    class_weights={  # Ajustar el peso de Williams
        'Williams': 2.0
    }
)




New https://pypi.org/project/ultralytics/8.3.78 available  Update with 'pip install -U ultralytics'
WARNING  'label_smoothing' is deprecated and will be removed in in the future.


SyntaxError: '[31m[1mclass_weights[0m' is not a valid YOLO argument. Similar arguments are i.e. ['classes'].

    Arguments received: ['yolo', '--f=c:\\Users\\victo\\AppData\\Roaming\\jupyter\\runtime\\kernel-v378f0546db9a8df102fd61cbab5be87f5ea0b4ec8.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of frozenset({'obb', 'segment', 'detect', 'pose', 'classify'})
                MODE (required) is one of frozenset({'train', 'track', 'export', 'predict', 'val', 'benchmark'})
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at 

In [13]:
# Evaluar después del fine-tuning
model_fine_tuned.val()

Ultralytics 8.3.69  Python-3.10.16 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 218 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\victo\Desktop\Documents\Tercer año\Segundo Cuatrimestre\Finales\f1-strategy\f1-dataset\valid\labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.14s/it]


                   all         20         29      0.643      0.762      0.839      0.729
           Kick Sauber          4          6      0.924      0.667      0.714      0.575
          Racing Bulls          1          1      0.202          1      0.995      0.796
                Alpine          4          5      0.822      0.931      0.928      0.888
               Ferarri          3          4      0.641       0.75      0.845      0.741
                  Haas          1          1      0.544          1      0.995      0.895
               Mclaren          5          6      0.755      0.516      0.805      0.738
              Mercedes          3          3          1      0.564      0.913      0.726
              Williams          2          3      0.253      0.667      0.519       0.47
Speed: 2.5ms preprocess, 26.3ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to ..\f1-strategy\yolo-files\runs\train_finetune2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 4, 5, 6, 7, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000024C2D9F1480>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047

In [15]:
# Guardar el modelo fine-tuned con un nuevo nombre
model_fine_tuned.save("../f1-strategy/weights/f1_detection_fine_tuned_best.pt")

print("Modelo fine-tuned guardado correctamente en: ../f1-strategy/weights/f1_detection_fine_tuned_best.pt")


Modelo fine-tuned guardado correctamente en: ../f1-strategy/weights/f1_detection_fine_tuned_best.pt


### Conclusiones del modelo fine tuned: Mejor usar el best o el best_fine_tuned?
Resultados del Modelo Anterior:
mAP50: 0.772
mAP50-95: 0.449
Precisión (P): 0.733
Recall (R): 0.615
Clases Problemáticas:

Kick Sauber: mAP50 ~0.619
Alpine: mAP50 ~0.556
Williams: mAP50 ~0.487
Resultados del Nuevo Modelo Fine-Tuned:
En Train:
mAP50: 0.840
mAP50-95: 0.728
Precisión: 0.643
Recall: 0.761
Clases Problemáticas:

Kick Sauber: mAP50 = 0.714
Alpine: mAP50 = 0.928
Williams: mAP50 = 0.522
En Validación:
mAP50: 0.839
mAP50-95: 0.729
Precisión: 0.643
Recall: 0.762
Clases Problemáticas:

Kick Sauber: mAP50 = 0.714
Alpine: mAP50 = 0.928
Williams: mAP50 = 0.519
Observaciones:
El mAP50 ha mejorado notablemente, subiendo de 0.772 a 0.840 en entrenamiento y 0.839 en validación.
El mAP50-95 también sube de 0.449 a 0.728 en entrenamiento y 0.729 en validación, lo cual indica un aumento en la precisión en diferentes umbrales de IoU.
La precisión ha subido ligeramente de 0.733 a 0.643 en el entrenamiento, pero con un recall mucho más alto, lo que sugiere que el modelo fine-tuned ha mejorado la capacidad de identificar los objetos relevantes, aunque el número de falsos positivos puede haber aumentado.
Las clases problemáticas como Kick Sauber, Alpine, y Williams han mostrado mejoras en mAP50, con un aumento significativo en Alpine (0.928 en validación, frente a 0.556 anteriormente).
Conclusión:
El modelo fine-tuned muestra una mejora general significativa, especialmente en mAP50 y en la capacidad para manejar clases problemáticas, aunque el modelo podría beneficiarse de un ajuste adicional en la precisión, especialmente para clases como Williams. Reducir las epochs de 40 a menos podría funcionar si se optimizan más hiperparámetros, pero parece que las 21 epochs han sido suficientes. 


In [ ]:
# Cargar el modelo entrenado para hacer predicciones
model = YOLO("f1_detection_best.pt")
model.to(DEVICE)

In [ ]:
# Evaluar en el conjunto de prueba
test_results = model.predict(source="test/images", save=True)

In [ ]:
import supervision as sv
# Función para detección en video
def detect_in_video(model, video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir el frame a RGB y hacer predicción
        results = model.predict(frame)
        
        # Dibujar los bounding boxes en la imagen
        annotated_frame = sv.draw_detections(frame, results[0].boxes)

        # Mostrar el video con detecciones
        cv2.imshow('F1 Detection', annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Ruta al video
video_path = "path/to/your/video.mp4"

# Detectar en el video
detect_in_video(model, video_path)